In [1]:
import pandas as pd
import numpy as np
import rasterio as rs
import geopandas as gpd
#from netCDF4 import Dataset
import os
import datetime
import sys
import gdal
from pyproj import Proj,transform
import osgeo.osr as osr

#reg = sys.argv[1] # pull the argument passed to the script
reg = '01'
fl = './data/nhrus/clean_AEA/nhru_%s_clean.shp'%reg # shapefile
idxraster = './data/NLDASv2_idx_125.tiff' # the index raster to use with

idxSRS = 'GEOGCS[\"unnamed ellipse\",DATUM[\"unknown\",SPHEROID[\"unnamed\",6371200,0]],PRIMEM[\"Greenwich\",0],UNIT[\"degree\",0.0174532925199433]]'
hruSRS = 'PROJCS["Lambert_Azimuthal_Equal_Area",GEOGCS["GCS_unnamed ellipse",DATUM["unknown",SPHEROID["Unknown",6370997,0]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",45],PARAMETER["longitude_of_center",-100],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["Meter",1]]'

def wkt2proj(wkt):
    srs = osr.SpatialReference()
    srs.ImportFromWkt(wkt)
    return srs.ExportToProj4()

idxSRS = wkt2proj(idxSRS)
hruSRS = wkt2proj(hruSRS)

idxSRS = Proj(idxSRS)
hruSRS = Proj(hruSRS)

In [130]:
def is_empty(any_structure):
    if any_structure:
        #print('Structure is not empty.')
        return False
    else:
        #print('Structure is empty.')
        return True
    
def get_cell(geom,gt=[],rb=[]):
    ''' Grab index cell value for an hru that is too small to create a tiff for.
    geom = hru geometry 
    gt = geotransform, of the index raster
    rb = raster band to pick data from
    '''
    # grab the centrioid
    mx = geom.centroid.x
    my = geom.centroid.y
    
    
    # convert the centroid back to geographic to pick the correct index cell
    mx,my = transform(p1=hruSRS,p2=idxSRS,x=mx,y=my)
    
    
    # transform to array coordinates
    px = int((mx-gt[0])/gt[1]) # x pixle
    py = int((my-gt[3])/gt[5]) # y pixle
    
    # extract the value
    intval = rb.ReadAsArray(px,py,1,1)
    return intval[0][0]

def process_tiffs(df,gt=[],rb=[]):
    '''Process the tiffs or use the cell finder if the tiff does not exist
    Inputs:
    df = geopandas data frame of the cooresponding regional shapefile
    gt = geotransform
    rb = raster band
    '''
    
    fl = './data/nhrus/spherical_tiffs/HUC_%s_nhruID_%s.tiff'%(df.region,df.hru_id_nat) # path to the tiffs
    #print(fl)
    percents = []
    cells = []
    cont=False
    
    if os.path.isfile(fl) == True: # only proceed if the tiff exists
        with rs.open(fl) as ds:
            rast = ds.read(1)
            nd = ds.nodata

        n,m = rast.shape
        rast.shape = n*m
        rast = rast[rast!=nd] # remove no data cells
        k = len(rast) # number of smaller cells in hru

        cells = list(np.unique(rast))
        #print(len(cells))
        if len(cells) > 0: # only continue if there are any cells
            percents = []
            for cell in cells:
                percents.append(len(rast[rast==cell])/k) # divide by the total cells in the basin to get the propotion of each cell in the basin

            cells = list(cells)
            cont=False
        else: 
            print('W - National HRU %s tiff empty, finding index value at centroid'%df.hru_id_nat)
            cells = [get_cell(df.geometry,gt = gt,rb = rb)]
            percents = [1.]
    
    elif (os.path.isfile(fl) == False): # if the raster does not exist, find the grid cell that the hru centroid occupies
        print('W - National HRU %s tiff missing, finding index value at centroid'%df.hru_id_nat)
        cells = [get_cell(df.geometry,gt = gt,rb = rb)]
        percents = [1.]
        
    #print(cells,percents)
    return cells,percents

def compute_contributions(fl,idxraster = idxraster):
    '''Compute grid cell contributions to hrus within a region'''
    
    reg = fl.split('_')[-2]
    
    print('S - Starting Region: %s!'%reg)
    print('S - Using input shapefile: %s'%fl)
    print('S - Using index raster: %s'%idxraster)
    
    # load the index raster and pull the geotransformation:
    src_ds = gdal.Open(idxraster)
    gt = src_ds.GetGeoTransform()
    rb = src_ds.GetRasterBand(1)

    dat = gpd.read_file(fl)
    
    cells,percents = zip(*dat.apply(process_tiffs,axis=1,gt=gt,rb=rb)) # run the aggregation function
    
    dat['reg'] = reg
    
    dat['cells'] = cells # insert results back into the dataframe
    dat['percents'] = percents
    
    # remove geometry from the data frame:
    del dat['geometry']
    outputFile = './data/NLDASv2_huc_%s_cell_contrib.pcl'%reg
    print('S - Writing contribution data to: %s'%outputFile)
    dat.to_pickle(outputFile)
    print('S - %s Complete!'%reg)

In [131]:
dat.head()

,POI_ID,hru_id_nat,hru_id_reg,region,geometry
0,7733855,1,1,01,POLYGON ((2183070.734501134 -75091.10246235332...
1,7733919,2,2,01,POLYGON ((2181894.320245298 -71005.04045235061...
2,7732571,3,3,01,POLYGON ((2178325.035636961 -69136.13575230232...
3,7732387,4,4,01,"POLYGON ((2179071.725215299 -69160.6980326779,..."
4,7733327,5,5,01,(POLYGON ((2189881.189432338 -69078.2122788688...


In [132]:
    idxraster = './data/NLDASv2_idx_125.tiff'
    fl = './data/nhrus/clean_AEA/nhru_01_clean.shp'
    reg = fl.split('_')[-2]
    
    print('S - Starting Region: %s!'%reg)
    print('S - Using input shapefile: %s'%fl)
    print('S - Using index raster: %s'%idxraster)
    
    # load the index raster and pull the geotransformation:
    src_ds = gdal.Open(idxraster)
    gt = src_ds.GetGeoTransform()
    rb = src_ds.GetRasterBand(1)

    dat = gpd.read_file(fl)
    
    cells,percents = zip(*dat.loc[dat.hru_id_nat == 14].apply(process_tiffs,axis=1,gt=gt,rb=rb)) # run the aggregation function

S - Starting Region: 01!
S - Using input shapefile: ./data/nhrus/clean_AEA/nhru_01_clean.shp
S - Using index raster: ./data/NLDASv2_idx_125.tiff
W - National HRU 14 tiff empty, finding index value at centroid


In [133]:
compute_contributions(fl)

S - Starting Region: 01!
S - Using input shapefile: ./data/nhrus/clean_AEA/nhru_01_clean.shp
S - Using index raster: ./data/NLDASv2_idx_125.tiff
W - National HRU 4 tiff missing, finding index value at centroid
W - National HRU 6 tiff missing, finding index value at centroid
W - National HRU 14 tiff empty, finding index value at centroid
W - National HRU 15 tiff missing, finding index value at centroid
W - National HRU 16 tiff missing, finding index value at centroid
W - National HRU 17 tiff empty, finding index value at centroid
W - National HRU 29 tiff missing, finding index value at centroid
W - National HRU 30 tiff empty, finding index value at centroid
W - National HRU 38 tiff empty, finding index value at centroid
W - National HRU 44 tiff empty, finding index value at centroid
W - National HRU 47 tiff missing, finding index value at centroid
W - National HRU 50 tiff empty, finding index value at centroid
W - National HRU 74 tiff missing, finding index value at centroid
W - Nationa

W - National HRU 1879 tiff missing, finding index value at centroid
W - National HRU 1898 tiff empty, finding index value at centroid
W - National HRU 1903 tiff missing, finding index value at centroid
W - National HRU 1941 tiff empty, finding index value at centroid
W - National HRU 1980 tiff missing, finding index value at centroid
W - National HRU 1988 tiff missing, finding index value at centroid
W - National HRU 2197 tiff empty, finding index value at centroid
W - National HRU 2208 tiff missing, finding index value at centroid
W - National HRU 2327 tiff missing, finding index value at centroid
W - National HRU 2359 tiff empty, finding index value at centroid
W - National HRU 2392 tiff empty, finding index value at centroid
S - Writing contribution data to: ./data/NLDASv2_huc_01_cell_contrib.pcl
S - 01 Complete!


In [134]:
test = pd.read_pickle('./data/NLDASv2_huc_01_cell_contrib.pcl')

In [135]:
test

,POI_ID,hru_id_nat,hru_id_reg,region,reg,cells,percents
0,7733855,1,1,01,01,"[44029, 44493]","[0.16666666666666666, 0.8333333333333334]"
1,7733919,2,2,01,01,"[44029, 44493]","[0.75, 0.25]"
2,7732571,3,3,01,01,[44029],[1.0]
3,7732387,4,4,01,01,[44029],[1.0]
4,7733327,5,5,01,01,[44030],[1.0]
5,7733755,6,6,01,01,[44030],[1.0]
6,7732571,7,7,01,01,[44029],[1.0]
7,7733923,8,8,01,01,"[44029, 44493]","[0.23076923076923078, 0.7692307692307693]"
8,7733327,9,9,01,01,[44030],[1.0]
9,7733755,10,10,01,01,[44030],[1.0]


In [17]:
dat = gpd.read_file(fl)

In [20]:
geom = dat.loc[dat.hru_id_nat == 5].geometry

In [22]:
geom.centroid

4    POINT (2188401.747535211 -69085.0446384551)
dtype: object

In [10]:
with rs.open('./data/nhrus/spherical_tiffs/HUC_01_nhruID_1.tiff') as ds:
    dat = ds.read(1)

In [11]:
ds.nodata

-9999.0

In [5]:
nd

NameError: name 'nd' is not defined

In [6]:
ds.nodata

9999.0

In [25]:
tmp= []

In [26]:
len(tmp)

0